# A-gs model and implementation (simulation CO2 and H2O flux)

## Initialize data and model

### Setup and fetch data

In [ ]:
# Settings
Username   = 'Beheerder'
years      = range(2001,2021)    #(1997,2021) # Set years to download

In [ ]:
import os
datapath   = os.path.join('../../') #since this file is 2 dir down from the data it's ../../
print('datapath is set to %s'%datapath)


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import plotly.express as px
#import cufflinks as cf
import matplotlib.dates as mdate
import matplotlib.ticker as ticker
from matplotlib import cm
#from colorspacious import cspace_converter
import scipy.stats as stats
#cf.go_offline()
# cf.set_config_file(offline=False, world_readable=True)

from datetime import datetime, timedelta
import sys
sys.path.insert(0, os.path.join(datapath,'PythonScripts'))
sys.path.append('../') #append the directory above this one to path to be able to find my python modules

from Loobos_Toolbox import dateparse, dateparse_Gapfilled, Read_LoobosEddFinal, Read_LooStor, Read_LoodatGapfill, Read_Loobos_halfhourly, Read_Loobos_meteo, Read_Loobos_soil, Read_Loobos_profile

from Ags_model import runAgs, runAgs2, calc_LE, calc_LE2

In [ ]:
import matplotlib.dates as mdates

In [ ]:
#these next two lines are to prevent re-loading the data. If you want to re-load data, instead comment them out
if not 'progress' in globals(): progress = list()
if not 'dataloaded' in progress:
  # Read files
    df_EC           = Read_LoobosEddFinal    (years,datapath)
    #df_Stor         = Read_LooStor           (years,datapath)
    df_Comb         = Read_LoodatGapfill     (years,datapath)
    #df_NEE          = Read_Loobos_halfhourly (years,datapath)
    df_meteo        = Read_Loobos_meteo      (years,datapath)
    df_soil         = Read_Loobos_soil       (years,datapath) 
    df_profile      = Read_Loobos_profile    (years,datapath)
    progress.append('dataloaded')

In [ ]:
# Make filter for GPP orginial data and not gapfilled
#General filters
I = ((df_Comb['GPP_fqc']==0)&(df_meteo['PAR']>0))
#t = df_profile.index                                          
#time = (t < np.datetime64('2013-05-08')) | (t > np.datetime64('2013-06-01'))

df_profile_filter = df_profile.copy()
df_meteo_filter   = df_meteo.copy()
df_Comb_filter    = df_Comb.copy()
df_EC_filter      = df_EC.copy()

# Filter for gpp original data and not gapfilled
#GPP_fcq_filt = ~(df_Comb['GPP_fqc']==0)
#df_Comb_filter.loc[GPP_fcq_filt] #this one removes all rows that match not just GPP_fqc

# Filter for CO2 data
CO2_filt = ~(df_profile['CO2level1'] > 300)
df_profile_filter.loc[CO2_filt,['CO2level1']] = np.nan

# Filter for PAR data
PAR_filt = ~(df_meteo['PAR'] >= 0)
df_meteo_filter.loc[PAR_filt,['PAR']] = np.nan

# Filter for L(o)corr data
Locorr_filt = ~((df_meteo['L(o)corr']>0) & (df_meteo['L(o)corr']<500)) 
df_meteo_filter.loc[Locorr_filt,'L(o)corr'] = np.nan

# Filter for L(o) data
Lo_filt = ~((df_meteo['L(o)']>0) & (df_meteo['L(o)']<40))
df_meteo_filter.loc[Lo_filt,'L(o)'] = np.nan

# Filter for Te-Lo data
TeLo_filt = ~(df_meteo['Te-L(o)']<40)
df_meteo_filter.loc[TeLo_filt,'Te-L(o)'] = np.nan

# Filter for VPD data
VPD_filt = ~(df_Comb['VPD']>=0)
df_Comb_filter.loc[VPD_filt,['VPD']] = np.nan

# Filter for U-star
Ustar_filt = ~(df_EC['U-star']>=0)
df_EC_filter.loc[Ustar_filt,['U-star']] = np.nan

# Import data from CSVs

### Import KNMI rain data

In [ ]:
#import KNMI rain data here. RD is daily sum rain.
df_rain = pd.read_csv('../knmi_rain_daily.csv')
df_rain.index = pd.to_datetime(df_rain['date'])
df_rain.drop('date',axis=1,inplace=True)
#df_rain

In [ ]:
df_rain

### Import fstr data

In [ ]:
#fstr max over 1 day, fstr values can be bigger than 1, day-values (08-18:00) and all
df_tmp1d      =pd.read_csv('../fstrOutput/NoRain/fstr_1d.csv')
df_tmp1d.index=pd.to_datetime(df_tmp1d['Unnamed: 0'])
df_tmp1d.index.name='date'
df_tmp1d.drop(columns='Unnamed: 0',inplace=True)
df_tmp1d_day  =pd.read_csv('../fstrOutput/NoRain/fstrMax_1d_day.csv')
df_tmp1d_day.index=pd.to_datetime(df_tmp1d_day['Unnamed: 0'])
df_tmp1d_day.index.name='date'
df_tmp1d_day.drop(columns='Unnamed: 0',inplace=True)

#fstr max over 1 day, values of fstr are clamped between 0 and 1, day-values (08-18:00) and all
df_tmp1dc     =pd.read_csv('../fstrOutput/NoRain/fstr_1d_clamped.csv')
df_tmp1dc.index=pd.to_datetime(df_tmp1dc['Unnamed: 0'])
df_tmp1dc.index.name='date'
df_tmp1dc.drop(columns='Unnamed: 0',inplace=True)
df_tmp1dc_day =pd.read_csv('../fstrOutput/NoRain/fstr_1d_clamped_day.csv')
df_tmp1dc_day.index=pd.to_datetime(df_tmp1dc_day['Unnamed: 0'])
df_tmp1dc_day.index.name='date'
df_tmp1dc_day.drop(columns='Unnamed: 0',inplace=True)

#fstr max over 1 day, values of fstr are clamped between 0 and 1, day-values (08:00-18:00) only
df_tmp1dcm    =pd.read_csv('../fstrOutput/NoRain/fstrMax_1d_clamped_day.csv')
df_tmp1dcm.index=pd.to_datetime(df_tmp1dcm['Unnamed: 0'])
df_tmp1dcm.index.name='date'
df_tmp1dcm.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
df_tmp1dc_day

# New graphs fstr over time

### define function for plotting fstr over time

In [ ]:
plt.rcParams["figure.figsize"] = (20,8)
def fstr_timeseries_wrapper(df_plot,ylims=None):
    df_plot = df_plot.loc[:,['fstr']].resample('7D').mean()
    #remove non-growthseason values
    df_plot=df_plot.loc[(df_plot.index.month>=4) & (df_plot.index.month<=9)]
    df_plot['weeknr']=df_plot.index.isocalendar().week
    
    fig,ax = plt.subplots()
    if ylims != None:
        ax.set_ylim(ylims[0],ylims[1])
    ax.set_ylabel('fstr (=LEobs/LEsim)')
    ax.set_xlabel('weeknr')
    
    plt.xticks([x for x in range(14,41)]) #this sets the xticks for the main x-axis with the weeknr
    locs, labels=plt.xticks() #this same function with no argument gets the position and labels and stores them for later
    
    ax2 = ax.twiny()
    fig.subplots_adjust(bottom=0.2)
    
    # Move twinned axis ticks and label from top to bottom
    ax2.xaxis.set_ticks_position("bottom")
    ax2.xaxis.set_label_position("bottom")
    
    # Offset the twin axis below the host
    ax2.spines["bottom"].set_position(("axes", -0.15))
    
    # Turn on the frame for the twin axis, but then hide all 
    # but the bottom spine
    ax2.set_frame_on(True)
    ax2.patch.set_visible(False)
    
    ax2.set_xlabel('approx month')
    ax2.set_xlim(13,41)
    ax2.set_xticks([14, 18, 22, 26,  30, 34, 39])
    ax2.set_xticklabels(['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'])
    
    colored_years={'2003':'green','2006':'orange','2007':'magenta','2011':'#b62020','2018':'blue','2019':'cyan'}
    
    for year in ['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2017','2018','2019']:
        start=year+'-01-01 01:00'
        end=year+'-12-23 23:00'
        label=str(year)
        if year in colored_years.keys():
            color=colored_years[year]
            marker='^'
            alpha=1.0
            linewidth=1.5
            
        else:
            color='grey'
            marker=''
            alpha=0.5
            label='_nolegend_'
            linewidth=1.0
        if year == str(2001):
            marker=''
            label='other years (2001-2017)'
        #marker={'2008':'^','2009':'^','2010':'^','2011':'^','2012':'o','2013':'^','2014':'o','2015':'^','2016':'o','2017':'o','2018':'^'}
        #colors={'2008':'#b62020','2009':'#cb2424','2010':'^','2011':'^','2012':'o','2013':'^','2014':'o','2015':'^','2016':'o','2017':'o','2018':'^'}
        #df_tmp.loc[start:end,'ET/LE'].plot(x=df_tmp.loc[start:end,'weeknr'],ylim=(0,15))
        ax.plot(df_plot.loc[start:end,'weeknr'],df_plot.loc[start:end,'fstr'], marker=marker,color=color,linewidth=linewidth,alpha=alpha ,label=label) #marker=marker[year]
    ax.legend(loc=("lower right"))
    ax.set_title('fstr (=LEobs/LEsim) on week-basis for 2001-2019 growth seasons (Apr-Sept), 3-hour means for 08:00-18:00, rainy days removed')
    #ax.set_title('fstr=0: witing point , fstr=1: no water stress')
    ax.grid()
    plt.show()

## weekly avg fstr over time (unclamped - fstr can be bigger than 1)

## weekly avg fstr over time (clamped - fstr between 0 and 1)

#### clamped but all hours go into weekly average (night too)

#### clamped but only daytime hours 08:00-18:00

In [ ]:
plt.rcParams.update({'font.size': 16})
fstr_timeseries_wrapper(df_tmp1dc_day,ylims=None)

In [ ]:
plt.rcParams.update({'font.size': 16})
fstr_timeseries_wrapper(df_tmp1dc_day,ylims=None)

## weekly mean of daily max fstr over time

In [ ]:
fstr_timeseries_wrapper(df_tmp1dcm,ylims=None)

## weekly max fstr over time

In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (20,6)

#remove non-growthseason values
df_plot = df_tmp1dcm.copy()
fstr_timeseries_wrapper(df_tmp1dcm,ylims=None)
df_plot = df_plot.loc[:,['fstr']].resample('7D').max()
df_plot=df_plot.loc[(df_plot.index.month>=4) & (df_plot.index.month<=9)]

df_plot['weeknr']=df_plot.index.isocalendar().week


fig,ax = plt.subplots()
#ax.set_ylim(0,1)
ax.set_ylabel('fstr (=LEobs/LEsim)')
ax.set_xlabel('weeknr')

plt.xticks([x for x in range(14,41)]) #this sets the xticks for the main x-axis with the weeknr
locs, labels=plt.xticks() #this same function with no argument gets the position and labels and stores them for later

ax2 = ax.twiny()
fig.subplots_adjust(bottom=0.2)

# Move twinned axis ticks and label from top to bottom
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")

# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.15))

# Turn on the frame for the twin axis, but then hide all 
# but the bottom spine
ax2.set_frame_on(True)
ax2.patch.set_visible(False)

ax2.set_xlabel('approx month')
ax2.set_xlim(13,41)
ax2.set_xticks([14, 18, 22, 26,  30, 34, 39])
ax2.set_xticklabels(['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'])

colored_years={'2003':'green','2006':'orange','2007':'magenta','2011':'#b62020','2018':'blue','2019':'cyan'}

for year in ['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2017','2018','2019']:
    start=year+'-01-01 01:00'
    end=year+'-12-23 23:00'
    if year in colored_years.keys():
        color=colored_years[year]
        marker='^'
    else:
        color='grey'
        marker=''
    #marker={'2008':'^','2009':'^','2010':'^','2011':'^','2012':'o','2013':'^','2014':'o','2015':'^','2016':'o','2017':'o','2018':'^'}
    #colors={'2008':'#b62020','2009':'#cb2424','2010':'^','2011':'^','2012':'o','2013':'^','2014':'o','2015':'^','2016':'o','2017':'o','2018':'^'}
    #df_tmp.loc[start:end,'ET/LE'].plot(x=df_tmp.loc[start:end,'weeknr'],ylim=(0,15))
    ax.plot(df_plot.loc[start:end,'weeknr'],df_plot.loc[start:end,'fstr'], marker=marker,color=color ,label=year) #marker=marker[year]
ax.legend(loc=("upper left"))
ax.set_title('fstr (=LEobs/LEsim) on week-basis for 2001-2020 growth seasons (Apr-Sept), 3-hour means for 09:00-15:00, rainy days removed')
#ax.set_title('fstr=0: witing point , fstr=1: no water stress')
ax.grid()

# Old way of Run A-gs model - Calculate ET - calculate and assemble fstr (obsolete, to be removed)

### Run A-gs model

In [ ]:
# Run A-gs model

an_final,an_umol,rs, ra = runAgs(df_profile_filter,df_Comb_filter,df_meteo_filter,df_EC_filter,fstr=1.0)

## Calcuate ET

### Assemble dataframe 'df_ET' that will hold output and fill with inputs

In [ ]:
def init_ETframe(rs_series):
    df_ET = pd.concat([df_meteo['L(o)corr'],df_profile['Pressure'],df_Comb['VPD'],df_Comb['rH'],df_meteo['P(mast)']],axis=1,sort=False)
    #convert Pressure from hPa to kPa 
    df_ET['p_kPa']=df_ET['Pressure']/10
    df_ET['VPD_adj']=df_ET['VPD'].loc[df_ET['VPD']>0] #some outlier values for VPD are negative, remove from dataset
    df_ET['VPD_adj']=df_ET['VPD_adj']/10  # VPD from df_Comb is in hPa, I need kPa, so hPa/10 = kPa
    df_ET['rs']=rs_series.to_frame(name="rs")
    df_ET['ra']=ra.to_frame(name="ra")
    return df_ET

In [ ]:
df_ET=init_ETframe(rs)

In [ ]:
df_ET=calc_LE2(df_ET)

In [ ]:
#df_ET

## get observed LE values

In [ ]:
df_LE_obs=df_Comb_filter[['LE']].copy()
LE_filt = ~(df_LE_obs['LE'] >= 0)
df_LE_obs.loc[LE_filt,['LE']] = np.nan
#df_ET_obs['LE'].plot()

## fstr analysis

### step 1 assemble dataframe and chop up according to each week

### calculate fstr

In [ ]:
#df_LE_obs = pd.read_csv('./LE_obs.csv')
#df_LE_obs.index = pd.to_datetime(df_LE_obs['datetime'])
#df_LE_obs.drop('datetime',axis=1,inplace=True)

df_LE_sim = pd.DataFrame()
df_LE_sim['ET'] = df_ET['ET_VPD']

df_fstr = pd.DataFrame()
df_fstr['LE'] = df_LE_obs['LE']
df_fstr=df_fstr.merge(df_LE_sim['ET'], how='inner',left_index=True, right_index=True)

df_fstr_3h = df_fstr.resample('3H').mean()

df_fstr_3h.loc[:,'LE/ET'] = df_fstr_3h['LE']/df_fstr_3h['ET']

#df_fstr

#### no longer used: this section can just be deleted

In [ ]:
#check same trim-size #obsolete
#print(df_fstr.index[0],df_fstr.index[0])
#print(df_rain_30m.index[0],df_rain_30m.index[-1])

In [ ]:
#st=df_fstr.index[0] #df_rain_30m needs to be trimmed to same size as df_fstr
#ed=df_fstr.index[-1]
#df_fstr_3h_filt = df_fstr_3h_filt.loc[df_rain_3h.loc[st:ed,'dailysum_rain']==0.0]

In [ ]:
df_rain_30m=df_rain.resample('30min').mean() #this is the wrong way to do it, but later on it's going to be summed)
df_rain['days_since_rain'].resample('30m').ffill() #overwrite wrong values

In [ ]:
df_rain_3h = df_rain_30m.resample('3H').sum()
df_rain_3h.loc[:,'days_since_rain'] = df_rain_30m['days_since_rain'].resample('3H').ffill()

In [ ]:
df_fstr_filt_3h=pd.concat([df_fstr_3h,df_rain_3h],axis=1)
df_fstr_filt_3h

In [ ]:
df_fstr_filt_1d = df_fstr_filt_3h.resample('1D').mean()
df_fstr_filt_7d = df_fstr_filt_3h.resample('7D').mean()

In [ ]:
df_fstr_filt_3h.loc[:,'LE/ET'] = df_fstr_filt_3h['LE']/df_fstr_filt_3h['ET']
df_fstr_filt_1d.loc[:,'LE/ET'] = df_fstr_filt_1d['LE']/df_fstr_filt_1d['ET']
df_fstr_filt_7d.loc[:,'LE/ET'] = df_fstr_filt_7d['LE']/df_fstr_filt_7d['ET']

### save fstr to file (is being done elsewhere, to be removed)

In [ ]:
#df_fstr_filt.to_csv('./fstrOutput/fstr_30m_norainydays.csv')
#df_fstr_filt_3h.to_csv('./fstrOutput/fstr_3H_norainydays.csv')
#df_fstr_filt_1d.to_csv('./fstrOutput/fstr_1D_norainydays.csv')
#df_fstr_filt_7d.to_csv('./fstrOutput/fstr_7D_norainydays.csv')

# Old way to visualize rain as bars and fstr as dots

In [ ]:
plt.rcParams["figure.figsize"] = (20,6)



def fstr_plot_wrapper(st,ed):

    fig,ax = plt.subplots()
    
    b=df_ET.loc[st:ed,['P(mast)']].resample('1D').sum()
    a=(df_rain.loc[st:ed,['RD']])
    ax.bar(b.index,b['P(mast)'],alpha=0.5, align='edge', label="Pmast data")
    ax.bar(a.index,a['RD'],alpha=0.5, align='edge', label="KNMI data")
    
    df_tmp30m=pd.DataFrame()
    df_tmp30m['ET']=df_ET['ET_VPD']
    df_tmp30m['P(mast)']=df_ET['P(mast)']
    df_tmp30m=df_tmp30m.dropna()
    
    df_LE=df_Comb['LE'].loc[df_Comb['LE']>=0]
    df_tmp30m=df_tmp30m.merge(df_LE, how='inner',left_index=True, right_index=True)
    c=df_tmp30m.loc[st:ed].resample('1D').sum()
    c['LE/ET']=c['LE']/c['ET']
    
    ax.set_ylabel("precip")
    ax2 = ax.twinx()
    ax2.set_ylim(0,5)
    ax2.set_ylabel("fstr")
    #ax2.set_yscale('log')
    
    ax2.scatter(c.index,c['LE/ET'],c=np.where(c['LE/ET']<1,'b','r'))
    
    ax.legend()
    ax.xaxis.set_minor_locator(mdates.DayLocator())

for year in range(2001,2021):
    st=str(year)+"-04-01"
    ed=str(year)+"-09-30"
    fstr_plot_wrapper(st,ed)

### Old way that I refactored (indistinguishable from old way)

In [ ]:
plt.rcParams["figure.figsize"] = (20,6)


def fstr_plot_wrapper(st,ed, df_fstr , df_ET , df_rain):

    fig,ax = plt.subplots()
    
    b=df_ET.loc[st:ed,['P(mast)']].resample('1D').sum()
    a=df_rain.loc[st:ed,['RD']]
    ax.bar(b.index,b['P(mast)'],alpha=0.5, align='edge', label="Pmast data")
    ax.bar(a.index,a['RD'],alpha=0.5, align='edge', label="KNMI data")

    df_plot=df_fstr.loc[st:ed,['LE','ET']]
    df_plot = df_plot.resample('1D').mean()
    df_plot.loc[:,'LE/ET']=df_plot['LE']/df_plot['ET']
    
    ax.set_ylabel("precip")
    ax2 = ax.twinx()
    ax2.set_ylim(0,5)
    ax2.set_ylabel("fstr")
    #ax2.set_yscale('log')
    
    ax2.scatter(df_plot.index,df_plot['LE/ET'],c=np.where(df_plot['LE/ET']<1,'b','r'))
    
    ax.legend()
    ax.xaxis.set_minor_locator(mdates.DayLocator())

for year in range(2001,2020):
    st=str(year)+"-04-01"
    ed=str(year)+"-09-30"
    fstr_plot_wrapper(st,ed , df_fstr_3h , df_ET , df_rain )

### end of visualization of rain as bars and fstr as dots

# Old graphs timeseries over fstr (redundant)

In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (20,6)

#remove non-growthseason values
df_plot = df_fstr_filt_7d.copy()
df_plot=df_plot.loc[(df_plot.index.month>=4) & (df_plot.index.month<=9)]

df_plot['weeknr']=df_plot.index.isocalendar().week


fig,ax = plt.subplots()
ax.set_ylim(0,2)
ax.set_ylabel('fstr (=LEobs/LEsim)')
ax.set_xlabel('weeknr')

plt.xticks([x for x in range(14,41)]) #this sets the xticks for the main x-axis with the weeknr
locs, labels=plt.xticks() #this same function with no argument gets the position and labels and stores them for later

ax2 = ax.twiny()
fig.subplots_adjust(bottom=0.2)

# Move twinned axis ticks and label from top to bottom
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")

# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.15))

# Turn on the frame for the twin axis, but then hide all 
# but the bottom spine
ax2.set_frame_on(True)
ax2.patch.set_visible(False)

ax2.set_xlabel('approx month')
ax2.set_xlim(13,41)
ax2.set_xticks([14, 18, 22, 26,  30, 34, 39])
ax2.set_xticklabels(['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'])

colored_years={'2003':'green','2006':'orange','2007':'magenta','2011':'#b62020','2018':'blue','2019':'cyan'}

for year in ['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2017','2018','2019']:
    start=year+'-01-01 01:00'
    end=year+'-12-23 23:00'
    if year in colored_years.keys():
        color=colored_years[year]
        marker='^'
    else:
        color='grey'
        marker=''
    #marker={'2008':'^','2009':'^','2010':'^','2011':'^','2012':'o','2013':'^','2014':'o','2015':'^','2016':'o','2017':'o','2018':'^'}
    #colors={'2008':'#b62020','2009':'#cb2424','2010':'^','2011':'^','2012':'o','2013':'^','2014':'o','2015':'^','2016':'o','2017':'o','2018':'^'}
    #df_tmp.loc[start:end,'ET/LE'].plot(x=df_tmp.loc[start:end,'weeknr'],ylim=(0,15))
    ax.plot(df_plot.loc[start:end,'weeknr'],df_plot.loc[start:end,'LE/ET'], marker=marker,color=color ,label=year) #marker=marker[year]
ax.legend(loc=("upper left"))
ax.set_title('fstr (=LEobs/LEsim) on week-basis for 2001-2020 growth seasons (Apr-Sept), 3-hour means for 09:00-15:00, rainy days removed')
#ax.set_title('fstr=0: witing point , fstr=1: no water stress')
ax.grid()

#### for some reason I re-made new dataframes to contain fstr and filter rain, but they dont work

In [ ]:
df_tmp30m=pd.DataFrame()
df_tmp30m['ET']=df_ET['ET_VPD']
df_tmp30m['P(mast)']=df_ET['P(mast)']
df_tmp30m=df_tmp30m.dropna()

df_LE=df_Comb['LE'].loc[df_Comb['LE']>=0]
df_tmp30m=df_tmp30m.merge(df_LE, how='inner',left_index=True, right_index=True)

#df_tmp30['ET/LE']=df_tmp30['ET']/df_tmp30['LE']
#df_tmp30m['LE/ET']=df_tmp30m['LE']/df_tmp30m['ET'] #<- this is the one we want, observations/model. If obs/model = fstr, then model*fstr= model(obs/model) = obs

df_tmp30m['P_3day']=df_tmp30m['P(mast)'].rolling('72H').sum()
#df_tmp30m=df_tmp30m.loc[df_tmp30m['P_3day']==0]

df_tmp3h=df_tmp30m.resample('3H').mean()#.between_time("09:00", "15:00")
df_tmp3h['P(mast)']=df_tmp30m['P(mast)'].resample('3H').sum() #overwrite the averaged pmast with sum pmast
df_tmp3h['P_3day']=df_tmp3h['P(mast)'].rolling('72H').sum() #overwrite the correct P_3day

#remove rainy days through knmi data
df_tmp3h['RD']=df_rain['RD']
df_tmp3h['RD']=df_tmp3h['RD']/10 #convert from tenths of mm to mm
df_tmp3h=df_tmp3h.loc[df_tmp3h['RD']==0]

#remove rainy days through Pmast
df_rain3h_dailymax=df_tmp3h['P(mast)'].resample('1D').sum().resample('3H').ffill()
df_rain3h_dailymax=df_rain3h_dailymax[:-1] #remove last entry because all 3h frames end on 21:00 not 00:00
df_rain3h_dailymax=df_rain3h_dailymax.rename('P(mast)Dmax')
df_tmp3h['P(mast)Dmax']=df_rain3h_dailymax
df_tmp3h=df_tmp3h.loc[df_tmp3h['P(mast)Dmax']==0]



#remove hours outside of 9:00 and 15:00
df_tmp3h=df_tmp3h.between_time("09:00", "15:00")

#resample to daily
df_tmp1d=df_tmp3h.resample('1D').mean()
df_tmp1d['doy']=df_tmp1d.index.dayofyear
df_tmp1d['P(mast)sum']=df_tmp3h['P(mast)'].resample('1D').sum()
df_tmp1d['LE/ET']=df_tmp1d['LE']/df_tmp1d['ET']

#resample to weekly (7 days)
df_tmp7d=df_tmp3h.resample('7D').mean()
df_tmp7d['LE/ET']=df_tmp7d['LE']/df_tmp7d['ET']

In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (20,6)

#remove non-growthseason values
df_plot = df_fstr_filt_7d.copy()
df_tmp7d=df_tmp7d.loc[(df_tmp7d.index.month>=4) & (df_tmp7d.index.month<=9)]

df_tmp7d['weeknr']=df_tmp7d.index.isocalendar().week

#df_tmp7d=df_tmp7d.loc[df_tmp7d['LE/ET']<=1.0]

#df_tmp

#df_tmp['ET/LE'].plot(ylim=(0,15))

fig,ax = plt.subplots()
ax.set_ylim(0,1.5)
ax.set_ylabel('fstr (=LEobs/LEsim)')
ax.set_xlabel('weeknr')

plt.xticks([x for x in range(14,41)]) #this sets the xticks for the main x-axis with the weeknr
locs, labels=plt.xticks() #this same function with no argument gets the position and labels and stores them for later

ax2 = ax.twiny()
fig.subplots_adjust(bottom=0.2)

# Move twinned axis ticks and label from top to bottom
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")

# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.15))

# Turn on the frame for the twin axis, but then hide all 
# but the bottom spine
ax2.set_frame_on(True)
ax2.patch.set_visible(False)

ax2.set_xlabel('approx month')
ax2.set_xlim(13,41)
ax2.set_xticks([14, 18, 22, 26,  30, 34, 39])
ax2.set_xticklabels(['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'])


for year in ['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2017','2018','2019']:
    start=year+'-01-01 01:00'
    end=year+'-12-23 23:00'
    if int(year)>=2010:
        marker='s'
    else:
        marker='o'

    ax.plot(df_tmp7d.loc[start:end,'weeknr'],df_tmp7d.loc[start:end,'LE/ET'], marker=marker ,label=year) #marker=marker[year]
ax.legend(loc=("upper left"))
fig.suptitle('fstr (=LEobs/LEsim) on week-basis for april t/m sept, 09:00-15:00, rainy days removed')
ax.set_title('fstr=0: witing point , fstr=1: no water stress')
ax.grid()

In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (20,6)

#remove non-growthseason values
df_tmp7d=df_tmp7d.loc[(df_tmp7d.index.month>=4) & (df_tmp7d.index.month<=9)]

df_tmp7d['weeknr']=df_tmp7d.index.isocalendar().week

#df_tmp7d=df_tmp7d.loc[df_tmp7d['LE/ET']<=1.0]

#df_tmp

#df_tmp['ET/LE'].plot(ylim=(0,15))

fig,ax = plt.subplots()
ax.set_ylim(0,1.5)
ax.set_ylabel('fstr (=LEobs/LEsim)')
ax.set_xlabel('weeknr')

plt.xticks([x for x in range(14,41)]) #this sets the xticks for the main x-axis with the weeknr
locs, labels=plt.xticks() #this same function with no argument gets the position and labels and stores them for later

ax2 = ax.twiny()
fig.subplots_adjust(bottom=0.2)

# Move twinned axis ticks and label from top to bottom
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")

# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.15))

# Turn on the frame for the twin axis, but then hide all 
# but the bottom spine
ax2.set_frame_on(True)
ax2.patch.set_visible(False)

ax2.set_xlabel('approx month')
ax2.set_xlim(13,41)
ax2.set_xticks([14, 18, 22, 26,  30, 34, 39])
ax2.set_xticklabels(['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'])

colored_years={'2003':'green','2006':'orange','2011':'#b62020','2013':'purple','2018':'blue'}

for year in ['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2017','2018','2019']:
    start=year+'-01-01 01:00'
    end=year+'-12-23 23:00'
    if year in colored_years.keys():
        color=colored_years[year]
        marker='^'
    else:
        color='grey'
        marker=''
    #marker={'2008':'^','2009':'^','2010':'^','2011':'^','2012':'o','2013':'^','2014':'o','2015':'^','2016':'o','2017':'o','2018':'^'}
    #colors={'2008':'#b62020','2009':'#cb2424','2010':'^','2011':'^','2012':'o','2013':'^','2014':'o','2015':'^','2016':'o','2017':'o','2018':'^'}
    #df_tmp.loc[start:end,'ET/LE'].plot(x=df_tmp.loc[start:end,'weeknr'],ylim=(0,15))
    ax.plot(df_tmp7d.loc[start:end,'weeknr'],df_tmp7d.loc[start:end,'LE/ET'], marker=marker,color=color ,label=year) #marker=marker[year]
ax.legend(loc=("upper left"))
fig.suptitle('fstr (=LEobs/LEsim) on week-basis for april t/m sept, 09:00-15:00, rainy days removed')
ax.set_title('fstr=0: witing point , fstr=1: no water stress')
ax.grid()

### end of old plots